# Imports

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import random as rd
from mlxtend.image import extract_face_landmarks
from imutils import paths

# A função draw_mask()

In [2]:
def draw_mask(img, ponto1=2, ponto2=15, ponto3=29, gray=80, random_color=True, color=False, padrao=None):
    '''Desenha máscara facial sobre nariz e boca detectada em imagem.

      Parâmetros
      ----------
      img : list
            Imagem sobre a qual será desenhada a máscara facial.
      ponto1 : int, optional
            Valor inteiro entre 0 e 67. Representa o landmark do lado esquerdo do rosto.
            default=2
      ponto2 : int, optional
            Valor inteiro entre 0 e 67. Representa o landmark do lado direito do rosto.
            default=15
      ponto3 : int, optional
            Valor inteiro entre 0 e 67. Representa o landmark da ponta do nariz.
            default=29
      gray : int, optional
            Valor inteiro entre 0 e 255. Representa a cor em escala de cinza da máscara que será desenhada sobre o rosto.
            default=80
      random_color : bool, optional
            Valor boolean que determina se a cor da máscara facial será aleatória.
            default=True
      color : bool, optional
            Valor boolean que determina se a máscara facial será colorida.
            default=False
      padrao : array, shape[h, w, ?]
            Imagem que determina qual será o padrão/textura que será desenhado sobre a máscara facial.
            default=None

      Retorno
      -------
      imagem : array, shape[h, w, ?]
            Mesma imagem da entrada, porém, com um desenho de poligono colorido/cinza
            no lugar onde a boca e nariz foram identificados.
    '''
  
    if random_color and color:
        color = [rd.randint(0,160),rd.randint(0,160),rd.randint(0,160)]
    elif random_color and color==False:
        gray=rd.randint(0,160)
        color = (gray, gray, gray)
    elif random_color == False:
        color = (gray, gray, gray)
  
    imagem = img.copy()
    landmark = extract_face_landmarks(imagem) # Retira os marcadores dos pontos do rosto para uma variavel em array
    if landmark is not None:
        pts1 = landmark[ponto1:ponto2]
        pts2 = np.array([landmark[ponto3], landmark[14], landmark[ponto1]], np.int32)
        pts = [pts1]+[pts2]

        if padrao is not None and img.shape == padrao.shape:
            mask = np.zeros_like(imagem,dtype=np.uint8)
            cv2.fillPoly(mask,pts,(255,255,255))
            inv_mask= cv2.bitwise_not(mask)
            new = cv2.bitwise_or(padrao,inv_mask)
            new = cv2.bitwise_not(new)

            cv2.fillPoly(imagem, pts, color) #desenha a máscara facial
            padronizado = cv2.bitwise_or(imagem,new)
            return padronizado
        else:
            cv2.fillPoly(imagem, pts, color) #desenha a máscara facial
            return imagem
    else:
        pass

# A célula a seguir se refere à preparação dos padrões/texturas que serão sobrepostos às máscaras artificiais.

In [3]:
caminho_padroes = 'padroes/padroes_originais/'
files_padroes = list(paths.list_images(caminho_padroes))
top, bottom, left, right = [150]*4

padroes = [cv2.imread(x, cv2.IMREAD_GRAYSCALE) for x in files_padroes]
a=1
for i in padroes:
  padrao = cv2.copyMakeBorder(i, top, bottom, left, right, cv2.BORDER_REFLECT)
  padrao = cv2.resize(padrao,(224,224))
  padrao = cv2.cvtColor(padrao, cv2.COLOR_GRAY2RGB)-200
  caminho0 = "padroes/padroes_reduzidos/" + str(a) + '.jpg'
  cv2.imwrite(caminho0, padrao)
  a+=1

# A célula a seguir se refere à execução da função draw_mask() no dataset e a separação de 50% de imagens com máscara e 50% sem máscara

In [4]:
caminho = 'datasets/dataset_reduzido/'
caminho_padroes = 'padroes/padroes_reduzidos/'
files = list(paths.list_images(caminho))
files_padroes = list(paths.list_images(caminho_padroes))
rd.shuffle(files)

padroes = [cv2.imread(x) for x in files_padroes]

num = len(files)
a=1

for file in files:
    i = cv2.imread(file)

    if a <= num*0.5:
        caminho = "datasets/dataset_final/com_mascara/" + str(a) + '.jpg'

        if a <= num*0.375:
            if i is not None:
                i = draw_mask(i, padrao=rd.choice(padroes))
        elif a > num*0.375:
            if i is not None:
                i = draw_mask(i, color=True, padrao=rd.choice(padroes))
    
    elif a > num*0.5:
        caminho = "datasets/dataset_final/sem_mascara/" + str(a) + '.jpg'

    if i is not None:
        cv2.imwrite(caminho, i)

    a+=1

C:\Users\jarth\anaconda3\envs\mascaras\lib\site-packages\mlxtend\image\extract_face_landmarks.py:66: UserWarning: No face detected.
  warnings.warn('No face detected.')
